In [96]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint

In [98]:
house_dict = []

for page in range(19, 21):
    linked = 'https://www.zillow.com/ca/duplex/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-127.94190190625,"east":-110.67139409375,"south":32.12531172436746,"north":42.36335181945167},"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"mapZoom":6,"filterState":{"isComingSoon":{"value":false},"isAuction":{"value":false},"isPreMarketForeclosure":{"value":false},"isPreMarketPreForeclosure":{"value":false},"isMakeMeMove":{"value":false},"isSingleFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isCondo":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            # Identify and return house data
            listing_type = str(result.a.get('aria-label').split(",")[0])
            price = result.find('div', class_="list-card-price").text
            full_address = result.find('h3', class_="list-card-addr").text
#             state = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            street = result.find('h3', class_="list-card-addr").text.split(",")[0]
            city = result.find('h3', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            try:
                state = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            house_type = loop_result1.select('li')[0].text.split(":")[1]
            year_build = loop_result1.select('li')[1].text.split(":")[1]
#             lot = loop_result1.select('li')[5].text.split(":")[1]
#             if (str(lot.split(" ")[1]) == "sqft"):
#                 lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""))
#             elif (str(lot.split(" ")[1]) == "acres"):
#                 lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', "")) * 43560
            try:
                zillow_price_per_sqft = loop_result1.select('li')[5].text.split(":")[1]
            except IndexError:
                zillow_price_per_sqft = 'null'
#             zillow_price_per_sqft = loop_result1.select('li')[5].text.split(":")[1]
            zestimate = loop_soup.find('span', class_="ds-estimate-value").text
            
            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "House_type": house_type,
                "Year_build": year_build,
#                 "Lot": lot,
#                 "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "Zestimate": zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            
        except AttributeError as e:
            continue

print(house_dict)

[{'Listing_type': 'Apartment for sale', 'Price': '$325,900', 'Full_address': '13222 Mohawk Rd APT 1, Apple Valley, CA 92308', 'Street': '13222 Mohawk Rd APT 1', 'City': 'AppleValley', 'State': 'CA', 'Zip_code': '92308', 'Bed': '7', 'Bath': '3', 'Square_Feet': '2,058', 'House_type': 'Multi Family', 'Year_build': '1952', 'Zestimate': '$322,927', 'Zillow_price/sqft': '$158', 'House_website': 'https://www.zillow.com/homedetails/13222-Mohawk-Rd-APT-1-Apple-Valley-CA-92308/2082511288_zpid/'}, {'Listing_type': 'Apartment for sale', 'Price': '$1,299,995', 'Full_address': '18257 Chatsworth St, Porter Ranch, CA 91326', 'Street': '18257 Chatsworth St', 'City': 'PorterRanch', 'State': 'CA', 'Zip_code': '91326', 'Bed': '6', 'Bath': '4', 'Square_Feet': '3,126', 'House_type': 'Multi Family', 'Year_build': '1959', 'Zestimate': '$1,270,809', 'Zillow_price/sqft': '$416', 'House_website': 'https://www.zillow.com/homedetails/18257-Chatsworth-St-Porter-Ranch-CA-91326/20160333_zpid/'}, {'Listing_type': 'Apa

In [99]:
house_data = pd.DataFrame(house_dict)
house_data

,Bath,Bed,City,Full_address,House_type,House_website,Listing_type,Price,Square_Feet,State,Street,Year_build,Zestimate,Zillow_price/sqft,Zip_code
0,3,7,AppleValley,"13222 Mohawk Rd APT 1, Apple Valley, CA 92308",Multi Family,https://www.zillow.com/homedetails/13222-Mohaw...,Apartment for sale,"$325,900","2,058",CA,13222 Mohawk Rd APT 1,1952,"$322,927",$158,92308
1,4,6,PorterRanch,"18257 Chatsworth St, Porter Ranch, CA 91326",Multi Family,https://www.zillow.com/homedetails/18257-Chats...,Apartment for sale,"$1,299,995","3,126",CA,18257 Chatsworth St,1959,"$1,270,809",$416,91326
2,3,6,Visalia,"3244 S Woodland St, Visalia, CA 93277",Multi Family,https://www.zillow.com/homedetails/3244-S-Wood...,Apartment for sale,"$319,900","2,483",CA,3244 S Woodland St,1984,"$308,675",$129,93277
3,--,Studio,Milpitas,"287/289 Marylinn Dr, Milpitas, CA 95035",Multi Family,https://www.zillow.com/homedetails/287-289-Mar...,Apartment for sale,"$1,150,000","1,860",CA,287/289 Marylinn Dr,1961,"$1,159,999",$618,95035
4,6,5,Pasadena,"326 -332 S Rosemead Blvd, Pasadena, CA 91107",Multi Family,https://www.zillow.com/homedetails/326-332-S-R...,Apartment for sale,"$1,388,000","4,846",CA,326 -332 S Rosemead Blvd,1936,"$1,361,996",$286,91107
5,17,Studio,Venice,"16 Thornton Ave, Venice, CA 90291",Multi Family,https://www.zillow.com/homedetails/16-Thornton...,Apartment for sale,"$5,000,000","9,546",CA,16 Thornton Ave,1913,"$4,879,567",$524,90291
6,4,5,Reseda,"19525 Elkwood St, Reseda, CA 91335",Multi Family,https://www.zillow.com/homedetails/19525-Elkwo...,Apartment for sale,"$849,990","2,077",CA,19525 Elkwood St,1953,"$849,992",$409,91335
7,8,9,Paramount,"8116 Howe St, Paramount, CA 90723",Multi Family,https://www.zillow.com/homedetails/8116-Howe-S...,Apartment for sale,"$1,200,000","3,685",CA,8116 Howe St,1966,"$1,198,697",$326,90723
8,4,3,LosAngeles,"5122 Huntington Dr S, Los Angeles, CA 90032",Multi Family,https://www.zillow.com/homedetails/5122-Huntin...,Apartment for sale,"$1,149,000","3,400",CA,5122 Huntington Dr S,1911,"$1,167,870",$338,90032
9,4,4,SantaPaula,"1142 Ojai Rd # 42, Santa Paula, CA 93060",Multi Family,https://www.zillow.com/homedetails/1142-Ojai-R...,Apartment for sale,"$790,000","2,046",CA,1142 Ojai Rd # 42,1971,"$779,210",$386,93060


In [100]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Q4-2019_Multi_Family_Data_page19-20.csv", index=False, header=True)